In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pathlib as pl
import sys
import subprocess

#### Define paths to swb2 executable and working/input data directories

In [ ]:
base_dir = str(pl.Path.cwd().parent)
python_script_dir = pl.Path.cwd().parent / 'python'
swb_work_dir = pl.Path.cwd().parent / 'swb_work'
logfile_dir = pl.Path.cwd().parent / 'logfiles'
test_data_dir = pl.Path.cwd().parent / 'test_data'
output_dir = pl.Path.cwd().parent / 'output'
swb2_executable = str(pl.Path.cwd().parent / 'bin' /'swb2.exe')
sys.path.append(str(python_script_dir))

from swb_cell import SWBCell
import run_management as rm

In [ ]:
# Show me all of the columns of the dataframe...
pd.set_option('display.max_columns', None)

#### Remove and replace working directories for swb2 run

In [ ]:
rm.destroy_model_work_output_and_logfile_dirs(base_dir=base_dir,
                                              swb_work_dir=swb_work_dir,
                                              logfile_dir=logfile_dir,
                                              output_dir=output_dir)
rm.create_model_work_output_and_logfile_dirs(base_dir=base_dir,
                                             swb_work_dir=swb_work_dir,
                                             logfile_dir=logfile_dir,
                                             output_dir=output_dir)


#### Make a new swb2 model run for a single cell example

In [ ]:
dry_run = False
lookup_dir_arg_text = f"--lookup_dir={str(test_data_dir)}"
weather_data_dir_arg_text = f"--weather_data_dir={str(test_data_dir)}"
output_dir_arg_text = f"--output_dir={str(output_dir)}"
logfile_dir_arg_text = f"--logfile_dir={str(logfile_dir)}"
control_file_path = str(test_data_dir / 'swb_control_file_kenai.ctl')
output_prefix = '--output_prefix=swb_kenai_'

swb_arg_text = [swb2_executable, output_dir_arg_text, lookup_dir_arg_text, weather_data_dir_arg_text, output_prefix, logfile_dir_arg_text, control_file_path]

f = open("stdout.txt", "w")

if not dry_run:
  with rm.cd(swb_work_dir):
#    p = subprocess.Popen(swb_arg_text, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    p = subprocess.Popen(swb_arg_text, stdout=f, stderr=subprocess.DEVNULL)
    p.wait()

#### Create a Python version of a swb2 cell

In [ ]:
# swb2 still uses Imperial units (inches and feet); the Python code expects metric units (mm and m).
# The example control file we're using for swb2 specifies 3.6 inches/foot for the available water capacity.
# 3.6 inches per foot equals 300 mm/m. The rooting depth is specified in the swb2 lookup table as 1.6404 feet, 
# or 1.6404 ft / 3.2808 ft/m = 0.5 m
mycell = SWBCell(latitude=60.57, available_water_capacity=300., rooting_depth=0.5)

#### Read in swb2 model results

In [ ]:
swb_df = pd.read_csv(pl.Path(output_dir, 'SWB2_variable_values__Kenai_Airpt__col_1__row_1__x_150393__y_1179331.csv'))
# eliminate pesky spaces around column names
swb_df.columns = swb_df.columns.str.replace(' ', '')
swb_df.columns

#### Convert swb Imperial units to metric

In [ ]:
swb_df['tminf'] = swb_df.loc[:,'tmin']
swb_df['tmaxf'] = swb_df.loc[:,'tmax']
swb_df['tmeanf'] = swb_df.loc[:,'tmean']
swb_df.loc[:,'tmin'] = (swb_df.loc[:,'tmin'] - 32) / 1.8
swb_df.loc[:,'tmax'] = (swb_df.loc[:,'tmax'] - 32) / 1.8
swb_df.loc[:,'tmean'] = (swb_df.loc[:,'tmean'] - 32) / 1.8
swb_df.loc[:,'gross_precip'] = swb_df.loc[:,'gross_precip'] * 25.4
swb_df.loc[:,'actual_ET'] = swb_df.loc[:,'actual_ET'] * 25.4
swb_df.loc[:,'reference_ET0'] = swb_df.loc[:,'reference_ET0'] * 25.4
swb_df.loc[:,'soil_storage'] = swb_df.loc[:,'soil_storage'] * 25.4
swb_df.loc[:,'snow_storage'] = swb_df.loc[:,'snow_storage'] * 25.4
swb_df.loc[:,'snowfall'] = swb_df.loc[:,'snowfall'] * 25.4
swb_df.loc[:,'snowmelt'] = swb_df.loc[:,'snowmelt'] * 25.4
swb_df.loc[:,'net_infiltration'] = swb_df.loc[:,'net_infiltration'] * 25.4

swb_df['date'] = pd.to_datetime(swb_df['date'])

Spurious LaTeX math example here

$$
f(x) = \int_{-\infty}^{\infty} e^{-x^2} dx
$$


#### Initialize a Python version of swb

In [ ]:
mycell.init_swb_cell()

#### Iterate and solve the water budget in Python over the date range contained in the swb2 output file

In [ ]:
for index, row in swb_df.iterrows():
    mycell.calc_cell_water_budget(int(row['year']),int(row['month']),int(row['day']),
                                  row['tmin'],row['tmax'],row['tmean'],row['gross_precip'])
    mycell.variables_todict()

#### Convert the output dictionary to a dataframe

In [ ]:
mycell.convert_dict_to_df()
py_df = mycell.output_df.copy()
py_df.loc[:,'date'] = pd.to_datetime(py_df['date'])

In [ ]:
df = pd.merge(left=swb_df, right=py_df, on='date', suffixes=('_s','_p'))
df.columns

In [ ]:
df['soil_storage_max']

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8),sharex=False)
ax.plot(df['date'], df['snowfall_s'], marker='.', linestyle=' ', color='blue', markersize=4, label='SWB snowfall')
ax.plot(df['date'], df['snowfall_p'], marker=' ', linestyle='-', 
        color='black', linewidth=0.75, label='Python model snowfall')
ax.legend()

In [ ]:
my_subset = df[(df['date'] > "1967-07-10") & (df['date'] < "1967-07-20")]
my_subset[['date','tmin','tmax','tmean','tminf','tmaxf','tmeanf','gross_precip','soil_storage_s','soil_storage_p',
           'reference_ET0','pet','actual_ET','aet']]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8),sharex=False)
ax.plot(df['date'], df['snowmelt_s'], marker='.', linestyle=' ', color='pink', markersize=4, label='SWB snowmelt')
ax.plot(df['date'], df['snowmelt_p'], marker=' ', linestyle='-', 
        color='black', linewidth=0.75, label='Python model snowmelt')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8),sharex=False)
ax.plot(df['date'], df['reference_ET0'], marker='.', linestyle=' ', color='cyan', markersize=4, label='SWB reference ET0')
ax.plot(df['date'], df['pet'], marker=' ', linestyle='-', 
        color='black', linewidth=0.75, label='Python model reference ET0')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8),sharex=False)
ax.plot(df['date'], df['snow_storage_s'], marker='.', linestyle=' ', color='purple', markersize=4, label='SWB snow storage')
ax.plot(df['date'], df['snow_storage_p'], marker=' ', linestyle='-', 
        color='black', linewidth=0.75, label='Python model snow storage')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8),sharex=False)
ax.plot(df['date'], df['soil_storage_s'], marker='.', linestyle=' ', color='green', markersize=4, label='SWB soil moisture')
ax.plot(df['date'], df['soil_storage_p'], marker=' ', linestyle='-', 
        color='black', linewidth=0.75, label='Python model soil moisture')
ax.legend()

In [ ]:
myplot = df.plot('date',['reference_ET0','actual_ET'], figsize=(12,4))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8),sharex=False)
ax.plot(df['date'], df['net_infiltration_s'], marker='.', linestyle=' ', color='orange', markersize=4, label='SWB net infiltration')
ax.plot(df['date'], df['net_infiltration_p'], marker=' ', linestyle='-', 
        color='black', linewidth=0.75, label='Python model net infiltration')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12, 8),sharex=False)
ax.plot(mycell.output_df['date'], df['actual_ET'], marker='.', linestyle=' ', color='green', markersize=4, label='SWB actual ET')
ax.plot(mycell.output_df['date'], mycell.output_df['aet'], marker=' ', linestyle='-', color='black',
        linewidth=0.75, label='Python model actual ET')
ax.legend()